# Data science

## Partie 1: Titanic

### Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

%matplotlib inline

### Util functions

In [ ]:
def count_bins(arr1, arr2):
    return len(set(arr1)), len(set(arr2))

### Import data from CSV

In [ ]:
titanic_data = np.genfromtxt('titanic.dat', delimiter=',', skip_header=1)

classes = titanic_data[:, 0]
ages = titanic_data[:, 1]
sexes = titanic_data[:, 2]
survived = titanic_data[:, 3]

### Class vs Age vs Sex

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig_3d = plt.figure()
ax = Axes3D(fig_3d)
ax.set_xlabel('class')
ax.set_ylabel('age')
ax.set_zlabel('sex')
ax.set_title('class vs age vs sex')
_ = ax.scatter(classes, ages, sexes)


### Class vs Age

In [ ]:
fig_class_age = plt.figure()
plt.xlabel('class')
plt.ylabel('age')
plt.title('class vs age')
_ = plt.hist2d(classes, ages, norm=LogNorm(), bins=count_bins(classes, ages))
_ = plt.colorbar()

### Class vs Sex

In [ ]:
fig_class_sex = plt.figure(3)
plt.xlabel('class')
plt.ylabel('sex')
plt.title('class vs sex')
_ = plt.hist2d(classes, sexes, norm=LogNorm())
_ = plt.colorbar()

### Age vs Sex

In [ ]:
fig_age_sex = plt.figure(4)
plt.xlabel('age')
plt.ylabel('sex')
plt.title('age vs sex')
_ = plt.hist2d(ages, sexes, norm=LogNorm(), bins=count_bins(ages, sexes))
_ = plt.colorbar()

### Recherche de K optimal

In [ ]:
from scipy.spatial import distance
from sklearn.cluster import KMeans

K = list(range(1, 50))
mean_dists = []
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(titanic_data[:,:3])
    
    dists = []
    for i, label in enumerate(kmeans.labels_):
        row = titanic_data[:,:3][i]
        cluster_center = kmeans.cluster_centers_[label]
        dist = distance.euclidean(row, cluster_center)
        dists.append(dist)
    
    mean_dist = np.mean(dists)
    mean_dists.append(mean_dist)

In [ ]:
plt.axes().set_yscale('linear')
_ = plt.plot(K, mean_dists)

In [ ]:
plt.axes().set_yscale('log')
_ = plt.plot(K, mean_dists)

## Partie 2

### Load breast cancer and wine datasets

In [ ]:
from sklearn import datasets
from pprint import pprint

breast_cancer = datasets.load_breast_cancer()
wine = datasets.load_wine()

### Normalization of data

In [ ]:
from sklearn import preprocessing



### Learning

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

breast_cancer_kneighbors = KNeighborsClassifier().fit(breast_cancer['data'], breast_cancer['target'])
breast_cancer_decisiontree = DecisionTreeClassifier().fit(breast_cancer['data'], breast_cancer['target'])
breast_cancer_MLPC = MLPClassifier().fit(breast_cancer['data'], breast_cancer['target'])

wine_kneighbors = KNeighborsClassifier().fit(wine['data'], wine['target'])
wine_decisiontree = DecisionTreeClassifier().fit(wine['data'], wine['target'])
wine_MLPC = MLPClassifier().fit(wine['data'], wine['target'])


### Validation croisée à 5 segments

## Partie 3

In [ ]:
news_data = np.genfromtxt('onlineNewsPopularity.csv', delimiter=',', skip_header=1)
nb_columns = np.shape(news_data)[1]
url, timedelta, n_tokens_title, n_tokens_content, n_unique_tokens, n_non_stop_words, \
n_non_stop_unique_tokens, num_hrefs, num_self_hrefs, num_imgs, num_videos, \
average_token_length, num_keywords, data_channel_is_lifestyle, data_channel_is_entertainment, data_channel_is_bus, \
data_channel_is_socmed, data_channel_is_tech, data_channel_is_world, kw_min_min, kw_max_min, kw_avg_min, \
kw_min_max, kw_max_max, kw_avg_max, kw_min_avg, kw_max_avg, kw_avg_avg, self_reference_min_shares, \
self_reference_max_shares, self_reference_avg_sharess, weekday_is_monday, weekday_is_tuesday, weekday_is_wednesday, \
weekday_is_thursday, weekday_is_friday, weekday_is_saturday, weekday_is_sunday, is_weekend, LDA_00, LDA_01, LDA_02, \
LDA_03, LDA_04, global_subjectivity, global_sentiment_polarity, global_rate_positive_words, \
global_rate_negative_words, rate_positive_words, rate_negative_words, avg_positive_polarity, min_positive_polarity, \
max_positive_polarity, avg_negative_polarity, min_negative_polarity, max_negative_polarity, title_subjectivity, \
title_sentiment_polarity, abs_title_subjectivity, abs_title_sentiment_polarity, shares = [news_data[:, i] for i in
                                                                                          range(nb_columns)]
